In [1]:
# Apagar a pasta caso existir
import shutil
shutil.rmtree('projeto-final', ignore_errors=True)
shutil.rmtree('sample_data', ignore_errors=True)

# Criar pasta e clonar o projeto
!mkdir projeto-final
!git clone https://github.com/franciscosens/bcc-processamento-imagens-trabalho-final projeto-final --quiet

# Copiar arquivos necessários
!cp projeto-final/requirements.txt requirements.txt
!cp projeto-final/setup.py setup.py
!cp projeto-final/setup.cfg setup.cfg
!cp projeto-final/domino.py domino.py
!cp -r projeto-final/mrcnn mrcnn

# Apagar projeto clonado
shutil.rmtree('projeto-final', ignore_errors=True)

In [2]:
%tensorflow_version 1.x
%matplotlib inline 

TensorFlow 1.x selected.


In [3]:
!pip install -r requirements.txt
!python setup.py install

     |████████████████████████████████| 17.3MB 28.0MB/s 
     |████████████████████████████████| 201.9MB 69kB/s 
     |████████████████████████████████| 276kB 42.8MB/s 
     |████████████████████████████████| 3.0MB 40.3MB/s 
     |████████████████████████████████| 163kB 43.2MB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=a5067b06b4d5ca60d5d5d7510d1b3361da9f6620393031f0714c75220842743e
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: umap-learn 0.4.4 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible

running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest file 'mask_rcnn.egg-info/SOURCES.txt'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.py -> build/bdist.linux-

In [4]:
# Apagar diretórios
import shutil
shutil.rmtree('dataset', ignore_errors=True)

# Criação dos diretórios necessários
!mkdir dataset
!mkdir dataset/train dataset/val

# Criação da pasta para armazenar os pesos da rede neural
!mkdir logs resultado

In [5]:
# Download das imagens de treinamento e de teste
!wget https://raw.githubusercontent.com/franciscosens/bcc-processamento-imagens-trabalho-final/master/dataset/val.zip -c --quiet
!wget https://raw.githubusercontent.com/franciscosens/bcc-processamento-imagens-trabalho-final/master/dataset/test.zip -c --quiet

# Descompactar imagens
!unzip -qq val.zip -d dataset/val/
!unzip -qq test.zip -d test/

!rm val.zip
! rm test.zip

In [6]:
from google.colab import drive
drive.mount('/content/drive')
!gsutil cp "/content/drive/My Drive/Processamento de Imagens Projeto Final/mask_rcnn_shapes3.h5" logs/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Copying file:///content/drive/My Drive/Processamento de Imagens Projeto Final/mask_rcnn_shapes3.h5...
-
Operation completed over 1 objects/244.0 MiB.                                    


In [7]:
from mrcnn.config import Config

class DominoConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "dom"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + domino

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [8]:
from domino import DominoDataset
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import json


config = DominoConfig()

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

custom_WEIGHTS_PATH = "logs/mask_rcnn_shapes3.h5"

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


custom_DIR = os.path.join(ROOT_DIR, "dataset")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Load validation dataset
dataset = DominoDataset()
dataset.load_domino(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

Using TensorFlow backend.



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

<module 'mrcnn.visualize' from '/content/mrcnn/visualize.py'>

In [9]:
from mrcnn import model as modellib, utils

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs/")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_shapes3.h5")



DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [10]:
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 89
Classes: ['BG', 'domino']


In [13]:
from domino import process_instances_and_return_image

# Load a random image from the images folder
try:
    from cv2 import cv2 as cv2
except ImportError:
    pass

folder_name = "test/test2"

IMAGE_DIR = os.path.join(ROOT_DIR, folder_name)
data_path = os.path.join(IMAGE_DIR ,'*g')
file_names =  glob.glob(data_path)

quantidade_total = len(file_names)
quantidade = 0
for file_name in file_names:
  posicao = file_name.rindex('/')
  
  file_name = file_name[posicao+1:len(file_name)]
  image = skimage.io.imread(os.path.join(IMAGE_DIR, file_name))

  # Run detection
  results = model.detect([image], verbose=1)

  # Visualize results
  r = results[0]
  imagem = process_instances_and_return_image(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'])
  cv2.imwrite('/content/resultado/' + file_name, imagem)
  quantidade = quantidade + 1
  print(f'{quantidade} de {quantidade_total}')

Processing 1 images
image                    shape: (192, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
1 de 822
Processing 1 images
image                    shape: (341, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
2 de 822
Processing 1 images
image                    shape: (192, 256, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.

In [14]:
!zip -r /content/resultado.zip /content/resultado
from google.colab import files
files.download("/content/resultado.zip")

updating: content/resultado/ (stored 0%)
updating: content/resultado/IMG_20200523_141522.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_140214.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_140108.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_140344.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_140420.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_135832.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_141444.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_135534.jpg (deflated 0%)
  adding: content/resultado/IMG_20200523_140521.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_131355.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_141345.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_135436.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_141408.jpg (deflated 1%)
  adding: content/resultado/IMG_20200523_141935.jpg (deflated 1%)
  adding: content/resultado/IMG_202

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>